<a href="https://colab.research.google.com/github/RodrigodeFeo/8DTS/blob/main/Aula_6_DTS_PLN_Demo_IA_Generativa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Demostração - Aula 6**

Classificando texto através de modelos generativos pré-treinados como BERT e GPT. Esses modelos são conhecidos como LLM´s (Large Language Model).

##**Modelo de classificação com Transformers usando o BERT**

In [1]:
!pip install transformers torch scikit-learn --quiet

In [2]:
# Importando as bibliotecas necessárias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import torch
from sklearn import preprocessing

In [3]:
# Carregando os dados
# Vamos usar nosso dataframe de produtos
df = pd.read_csv("https://dados-ml-pln.s3-sa-east-1.amazonaws.com/produtos.csv", delimiter=";", encoding='utf-8')
df.dropna(inplace=True)
df["texto"] = df['nome'] + " " + df['descricao']
df = df.sample(1000, random_state=42)
print(df.info())

# Certifique-se de ter os dados pré-processados e tratados antes desta etapa
# Aqui, estou usando dados com uma pequena amostra para efeito de demostração

# Dividindo os dados em conjuntos de treinamento e teste
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 2700 to 992
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   nome       1000 non-null   object
 1   descricao  1000 non-null   object
 2   categoria  1000 non-null   object
 3   texto      1000 non-null   object
dtypes: object(4)
memory usage: 39.1+ KB
None


In [4]:
# Tokenizando os textos

# Inicializando o tokenizador BERT para o português
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

# Seperação dos dados de treino e teste
## Dados de treino
train_texts = train_df['texto'].tolist()
train_labels = train_df['categoria'].tolist()
## Dados de teste
test_texts = test_df['texto'].tolist()
test_labels = test_df['categoria'].tolist()

# Codificando os textos usando o tokenizador
train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors='pt', max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors='pt', max_length=128)

# Codificando as categorias em números e convertendo as categorias para IDs numéricos
le = preprocessing.LabelEncoder()
train_labels = le.fit_transform(train_labels)
test_labels = le.transform(test_labels)

# Convertendo os dados para tensores PyTorch
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(train_labels))
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], torch.tensor(test_labels))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

Tensores são estruturas de dados fundamentais em muitos frameworks de computação numérica, incluindo o PyTorch. Eles são generalizações de matrizes e são usados para representar dados multi-dimensionais. Em NPL usamos para criar embeddings com mais características.

In [5]:
# Inicializando o modelo BERT pré-treinado para classificação
model = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased', num_labels=len(set(train_labels)))

# Inicializando o otimizador
optimizer = AdamW(model.parameters(), lr=5e-5)

# Inicializando o DataLoader para treinamento e teste
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# Treinamento do modelo
## identifica qual o processador da sessão (CPU ou GPU?), recomendo usar GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

num_epochs = 3

for epoch in range(num_epochs):
    for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}"):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 3: 100%|██████████| 175/175 [00:18<00:00,  9.64it/s]


*   input_ids: É uma sequência de IDs de tokens que representa a entrada do modelo. Cada token em um texto é convertido para um ID específico usando o tokenizador BERT.
*   attention_mask: É uma máscara de atenção que informa ao modelo quais tokens devem ser considerados (1) e quais devem ser ignorados (0). Isso é útil para lidar com sequências de comprimentos variados.
*   labels: São as etiquetas verdadeiras (rótulos) associadas a cada exemplo no lote. No contexto de um problema de classificação, como este, as labels indicam a categoria correta para cada exemplo.


In [6]:
'''
for batch in train_loader:
    input_ids, attention_mask, labels = batch
    # Aqui você realiza as operações de treinamento usando o lote de dados
'''

'\nfor batch in train_loader:\n    input_ids, attention_mask, labels = batch\n    # Aqui você realiza as operações de treinamento usando o lote de dados\n'

In [7]:
# Avaliação do modelo
model.eval()
all_predictions = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating"):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)

        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Mapeando os IDs preditos de volta para as categorias originais usando inverse_transform
#predicted_labels = all_predictions
#true_labels = all_labels
predicted_labels = le.inverse_transform(all_predictions)
true_labels = le.inverse_transform(all_labels)

# Exibindo as métricas de classificação
print(classification_report(true_labels, predicted_labels))

Evaluating: 100%|██████████| 75/75 [00:02<00:00, 34.29it/s]

              precision    recall  f1-score   support

   brinquedo       0.76      1.00      0.86        69
        game       1.00      0.95      0.97        60
       livro       1.00      0.76      0.86        83
   maquiagem       0.99      1.00      0.99        88

    accuracy                           0.92       300
   macro avg       0.94      0.93      0.92       300
weighted avg       0.94      0.92      0.92       300



##**Geração de texto via API ChatGPT**

Funciona para classificação? Simulando um prompt para classificxar um texto

In [8]:
!pip install openai --quiet
#!pip install --upgrade openai --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00


In [9]:
import openai
print(openai.__version__)

1.30.1


In [10]:
'''
Documentação:
  GitGub = https://github.com/openai/openai-python
  Início rápido do desenvolvedor - API = https://platform.openai.com/docs/quickstart?context=python
    Exemplos: https://platform.openai.com/docs/api-reference/authentication
  Limites e taxas = https://platform.openai.com/docs/guides/rate-limits/usage-tiers?context=tier-free
  Outros: https://github.com/openai/openai-python/discussions/742
'''

'\nDocumentação:\n  GitGub = https://github.com/openai/openai-python\n  Início rápido do desenvolvedor - API = https://platform.openai.com/docs/quickstart?context=python\n    Exemplos: https://platform.openai.com/docs/api-reference/authentication\n  Limites e taxas = https://platform.openai.com/docs/guides/rate-limits/usage-tiers?context=tier-free\n  Outros: https://github.com/openai/openai-python/discussions/742\n'

In [12]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

!ls -la /content/gdrive/MyDrive/FIAP/NLP/setup_api_openai.py
!cp /content/gdrive/MyDrive/FIAP/NLP/setup_api_openai.py /content
!ls -la /content

Mounted at /content/gdrive
ls: cannot access '/content/gdrive/MyDrive/FIAP/NLP/setup_api_openai.py': No such file or directory
cp: cannot stat '/content/gdrive/MyDrive/FIAP/NLP/setup_api_openai.py': No such file or directory
total 20
drwxr-xr-x 1 root root 4096 May 18 17:09 .
drwxr-xr-x 1 root root 4096 May 18 16:47 ..
drwxr-xr-x 4 root root 4096 May 16 13:24 .config
drwx------ 5 root root 4096 May 18 17:08 gdrive
drwxr-xr-x 1 root root 4096 May 16 13:24 sample_data


In [13]:
#API key
#OPENAI_API_KEY = "xxxxx"

In [16]:
from openai import OpenAI
import setup_api_openai as api

OPENAI_API_KEY = api.OPENAI_API_KEY

client = OpenAI(
    api_key=OPENAI_API_KEY,
)

ModuleNotFoundError: No module named 'setup_api_openai'

In [15]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "Você é um modelo de classificação de texto no contexto de produto, gostaria que classificasse o texto abaixo entre as categorias: maquiagem, livro, brinquedo e game. O resultado gostaria que seguisse o template de output: {"'"texto"'": <'texto original do imput'>, "'"categoria"'": <'categoria classificada pelo chatgpt'>}."},
    {"role": "user", "content": "Texto: 'Maleta Exclusiva Ruby Rose Profissional Completa.'"}
  ]
)

print(completion.choices[0].message)
print(completion.choices[0])

NameError: name 'client' is not defined

In [ ]:
resposta = completion.choices[0].message.content
print(resposta)

{"texto": "Maleta Exclusiva Ruby Rose Profissional Completa.", "categoria": "maquiagem"}


In [ ]:
type(resposta)

str

In [ ]:
#!pip install --upgrade jsonlib --quiet

In [ ]:
import json

#convert string to object
json_resposta = json.loads(resposta)
print(type(json_resposta))

<class 'dict'>


In [ ]:
json_resposta["categoria"]

'maquiagem'